In [2]:
import pandas as pd
import numpy as np
import jinja2
from xhtml2pdf import pisa
import plotly.express as px
import plotly.graph_objects as go
#for worldmap
from urllib.request import urlopen
import json

ModuleNotFoundError: No module named 'xhtml2pdf'

# PDF Report
## Preparation
Firstly, we will create a HTML-report with jinja2. Afterwards, this html-file will be converted to pdf by pdfkit.

In [7]:
# Template handling
env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=''))
template = env.get_template('report_template.html')


In [8]:
#Define document colors: 0:Green, 1:yellow, 2:red, 3:beige, 4:grey, 5:light-green, 6:light-yellow, 7:light-red, 8:dark-beige, 9: light-grey
colors = ['#1D7151', '#E39617', '#CC0000', '#E3DCCA', '#505C5D', '#8FBF99', '#F2C279','#F2AA91', '#CCAD99', '#ACBFBF']
font='Roboto-light, Gravitas One, Raleway, Arial, sans-serif'
title_font='Montserrat-Regular, Raleway, sans-serif'

## Introduction

In [9]:
introduction = '''Der Klimawandel ist aktueller denn je. Um diesem entgegen zu wirken, müssen Treibhausgasemissionen und negative Einflüsse auf unsere Umwelt reduziert oder gänzlich vermieden werden.
                Die <b><font color='#1D7151'>Lebensmittelindustrie</font></b> ist ein wesentlicher Treiber dieser Umweltschäden. Eine bewusste Auswahl von Lebensmitteln und eine gesunde Ernährung sind eine wirksame Möglichkeit, um auch als Endverbraucher die Nachhaltigkeit dieses Sektors zu fördern. 
                In diesem Bericht gehen wir auf die <b>Umweltauswirkungen</b> der Lebensmittelindustrie ein. Insbesondere zeigen wir den <b>Ursprung</b> und die <b>Entwicklung</b> der Treibhausgasemissionen. 
                Des Weiteren zeigen wir den Umwelteinfluss einzelner Lebensmittel auf, geben eine <b>Ernährungsempfehlung</b> und vergleichen diese mit der durchschnittlichen Ernährungsweise einzelner Länder.
                '''

## 1 Environmental Impact of the food industry
### Figure 1.1 Different impacts

In [10]:
#text
fig_1_1_title='Wie wirkt sich die Lebensmittelindustrie auf die Umwelt aus?'
fig_1_1_description='''Die Grafik zeigt, welchen Anteil die Lebensmittelindustrie an verschiedenen, negativen Umweltauswirkungen besitzt. So ist die Lebensmittelindustrie z.B. für <b>ein Drittel der weltweiten Treibhausgasemissionen</b> verantwortlich. <br>
                    Für den Anbau und die Tierhaltung werden <b>50% der weltweit nutzbaren Fläche</b> beansprucht. Die Anbauflächen werden dabei überwiegend für die Futtermittelproduktion der Tiere (77%) und nicht für direkte Nahrungsmittel für den Menschen genutzt.<br>
                    Die andere Hälfte umfasst menschlichen Wohnraum, Wälder und Süßwasser. <br>Des Weiteren werden für die Bewässerung und Tierhaltung große Mengen Wasser benötigt. <b>70% des weltweiten Süßwasserverbrauchs</b> entsteht durch die Landwirtschaft, Industrie und Privathaushalte verbrauchen im Vergleich dazu nur 19 % bzw. 11%.
                    <br>Ein weiterer wichtiger, negativer Einfluss auf unsere Umwelt ist die Eutrophierung. Eutrophierung bezeichnet die Verschmutzung von Gewässern in Form von schädlichem Pflanzenwachstum durch den Eintrag nährstoffreicher Schadstoffe. Hauptursache für diese Verschmutzung ist die Überdüngung von Böden in der Landwirtschaft.
                    <br>Außerdem beeinflusst die Landwirtschaft auch die Artenvielfalt. 94% aller Säugetiere (Menschen ausgeschlossen) sind Nutztiere. Das heißt <b>nur 6% der Säugetiere leben in freier Wildbahn.</b>'''

In [11]:
#figure
df_impact_total = pd.read_csv('../Dashboard/fig1_environmental_impact_food.csv', delimiter=';')
df_impact_total['impact']=df_impact_total['impact'].astype('category')
df_impact_total['sector']=df_impact_total['sector'].astype('category')
df_impact_total['description']=df_impact_total['description'].astype('category')



fig_1_1=go.Figure([
        #Bar 1 : Lebensmittelindustrie
        go.Bar(name='Lebensmittelindustrie', 
                x=df_impact_total['impact'], 
                y=df_impact_total['percent'],
                
                customdata=df_impact_total['sector'],
                #hide hovertext
                hovertext=None,
                hoverinfo='skip',
                marker_color= [colors[0], colors[3],colors[0], colors[3],colors[0], colors[3],colors[0], colors[3],colors[0], colors[3]],
                text = df_impact_total["percent"],
                texttemplate='%{text:p}',
                showlegend=False,
                textposition='inside'
                     
                ),
        
])
fig_1_1.update_layout(
                barmode='stack', #creates stacked plot instead of two bars next to each other
                # Label axis + title
                title={'text':"Anteil der <b style='color:#1D7151'>Lebensmittelindustrie</b> an verschiedenen Umweltschäden",'font_family':'Montserrat-Regular, Raleway, sans-serif', 'x':0, 'y':1},
                #xaxis = {"title": "Art des Umwelteinflusses"},
                yaxis={'tickformat':'.0%'},
                font_family=font,
                #legend={'orientation':'h', 'yanchor':"bottom",'y':1.02,'xanchor':"right",'x':1},
                margin=dict(t=30, r=10, l=10, b=10)
            )


In [12]:
fig_1_1.write_image("fig_1_1.png", width='600', scale=1.5)

### Figure 1.2 GHG emissions along the production chain

In [13]:
#text
fig_1_2_title='Wie entstehen die Treibhausgasemissionen in der Landwirtschaft?'
fig_1_2_description='''Da Energie vor allem in der Industrie ebenfalls ein wichtiger Treiber des Klimawandels ist, konzentrieren sich die aktuellen Gegenmaßnahmen hauptsächlich auf den Einsatz erneuerbarer Energien, einer Steigerung der Energieeffizienz und einer kohlenstoffarmen Verkehrsalternative. 
                Nachgelagerte Prozesse wie die Verarbeitung, Verpackung und Transport der Lebensmittel können durch die Weiterentwicklungen im Energiebereich reduziert werden. Jedoch sind <font color='#CC0000'>nachgelagerte Prozesse z.B. Verpackung und Transport</font> nur einen geringen Teil (15%) der Emissionen in der Lebensmittelindustrie verantwortlich. 
                Dies widerlegt auch die verbreitete Annahme, dass regionale Lebensmittel die Lösung für eine nachhaltige Ernährung sind.<br>
                Der Großteil der Treibhausgasemissionen entsteht hier nämlich in der <font color='#E39617'>Landwirtschaft</font> (39%) und durch die <font color='#1D7151'>Flächennutzung</font> (32%). Hier gibt es aktuell noch keine praktikablen Lösungen, um diesen Emissionen entgegen zu wirken. 
                Dünger ist unersetzlich, um den wachsenden Nahrungsmittelbedarf zu decken und die Methanproduktion von Rindern kann nicht verhindert werden. Deswegen benötigen wir vor allem eine Verbesserung der landwirtschaftlichen Effizienz und neue Technologien, um kohlenstoffarme Lebensmittel skalierbar und erschwinglich zu machen.<br>
               <font color='#CCAD99'> Nach dem Verkauf</font> an den Endkunden entstehen weitere Emissionen hauptsächlich in Form von Abfall. Als Endverbraucher ist die Vermeidung von Lebensmittelverschwendung eine gute Möglichkeit, um die Emissionen zu senken.'''

In [14]:
#figure
df_impact_steps = pd.read_csv('../Dashboard/fig2_emissions-food-per-step.csv', delimiter=';')

#Change Datatypes
df_impact_steps['step']=df_impact_steps['step'].astype('category')
df_impact_steps['detail']=df_impact_steps['detail'].astype('category')

#Add constant column
#df_impact_steps['sector']='Food Industry'

fig_1_2=go.Figure([
        #Bar 1 : Lebensmittelindustrie
        go.Pie(values=df_impact_steps['percentage'], 
                labels=df_impact_steps['detail'].str.wrap(25).apply(lambda x: x.replace('\n', '<br>')),
                #text
                insidetextorientation='horizontal',
                showlegend=False,
                sort=False,
                customdata=df_impact_steps['step'],
                #hovertext
                hovertext=df_impact_steps['detail'].str.wrap(25).apply(lambda x: x.replace('\n', '<br>')),
                hovertemplate="<b>%{customdata[0]}</b><extra></extra>"
                )
])

#Styling
fig_1_2.update_traces(textinfo='label+percent', 
    marker={'colors': [colors[0], colors[1], colors[2], colors[2], colors[2], colors[2], colors[3], colors[3]],
    'line':{'color': '#fff', 'width': 1}},
    textfont_family=font,
    )
fig_1_2.update_layout(
            title={'text':"Ursprung der Treibhausgasemissionen der Lebensmittelindustrie",'font_family':'Montserrat-Regular, Raleway, sans-serif', 'x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10)
            )

In [15]:
fig_1_2.write_image("fig_1_2.png", width='600', scale=1.5)

### Figure 1.3 Global distribution of GHG Emissions by food production

In [16]:
#text
fig_1_3_title='Die Treibhausgasemissionen der Länder unterscheiden sich stark.'
fig_1_3_description='''Die Grafik zeigt die Emissionen jedes Landes, die bei der Produktion von Lebensmitteln entstanden sind und nicht, wo diese Lebensmittel dann tatsächlich konsumiert wurden. Je dunkler ein Land, desto mehr Emissionen entstehen dort.
                    Das bedeutet, wenn Brasilien Regenwald abrodet, um Fleisch für die USA zu produzieren, werden die Emissionen Brasilien zugeordnet. 
                    Auf diese Weise melden die Länder ihre Emissionen an die Vereinten Nationen und legen Klimaziele fest.<br>
                    Fünf Staaten haben einen Anteil <b>über 5%</b> an den globalen Emissionen der Lebensmittelindustrie: <font color='#B35900'>China (13,8%/2,42 Mrd. t), Indonesien (8,8%/1,58 Mrd. t), USA (8,2%/1,47 Mrd. t), Brasilien (7,4%/1,32 Mrd. t) und Indien (6,3%/1,13 Mrd. t)</font>. <br>
                    Diese Länder waren zu erwarten, da sie sich entweder durch eine hohe Bevölkerungszahl und/oder einen großen Agrarsektor auszeichnen. Allerdings gibt es eine große Lücke zwischen diesen Ländern und dem Rest der Welt. Nach Indien folgt erst wieder Russland mit 465 Mio. t (2,6%).'''

In [17]:
#preparation
df_world = pd.read_csv('../Dashboard/fig3_global_ghg_emissions.csv')
#Change datatypes
df_world['Entity']=df_world['Entity'].astype('category')
df_world['Code']=df_world['Code'].astype('category')

#load geojson
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
         world = json.load(response)

In [18]:
#figure
df_world_total_sub=df_world[(df_world['Year'] == df_world['Year'].max()) & (~df_world['Entity'].isin(['World', 'Developing', 'Industrialized']))]

colorscale = [(0.00, '#F2E6C2'), (0.16, '#F2E6C2'),
                (0.16, '#F2C279'), (0.32, '#F2C279'),
                (0.32, '#F2AA3D'),  (0.48, '#F2AA3D'),
                (0.48, '#E39617'), (0.64, "#E39617"),
                (0.64, '#CC7A14'),  (0.83, '#CC7A14'),
                (0.83, '#B35900'),(1.0,'#B35900')]
mapboxtoken="pk.eyJ1Ijoic2UwODkiLCJhIjoiY2t5YTJhdGJkMDFlNTJvcDh5dnMwc2xhMCJ9.oKhU7ogigoJsOEG85QHv3Q"
mapboxstyle="mapbox://styles/se089/ckya2d29s34ma16n2v9x5mz6g"
fig_1_3= px.choropleth_mapbox(
        df_world_total_sub,
        geojson=world,
        locations=df_world_total_sub['Code'],
        #colorscale
        color=np.log10(df_world_total_sub['total_food_emissions'].astype(float)),
        color_continuous_scale=colorscale,
        range_color=(0, 3000000000),
        
        #custom data for hover
        custom_data= [df_world_total_sub['Entity'], df_world_total_sub['total_food_emissions'], df_world_total_sub['share_food_emissions']],
        
        #position map
        mapbox_style=dict(accesstoken=mapboxtoken,
                          center=dict(lat=-20, lon=-20),
                          style='light', # change mapbox style
                          zoom=1),
        zoom=0.7,
        center={'lat': 10, 'lon': 10},
        opacity=0.8,
        labels=df_world_total_sub['Entity'],
        
    )
#customize colorbar, change number formatting
fig_1_3.update_coloraxes(colorbar_x=-0.1,
    
    colorbar_tickfont=dict(family=font)
    )

fig_1_3.update_traces(
    z=df_world_total_sub['total_food_emissions'].astype(float), zmin=0,
    zmax=df_world[~df_world['Entity'].isin(['World', 'Developing', 'Industrialized'])]['total_food_emissions'].max(),
    
    hovertemplate="<b>%{customdata[0]}:</b><br>Emissionen: %{customdata[1]:,r} t</br>Anteil an globalen Emissionen: %{customdata[2]:.2%} "

)
fig_1_3.update_layout(coloraxis_colorbar=dict(
    title="Emissionen <br> in t",
    tick0=0,
    dtick=500000000,
    titleside='bottom'
    ),
    title={'text':"Globale Treibhausgasemissionen der Landwirtschaft in t",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10),
    mapbox={"accesstoken":mapboxtoken, 'style':'white-bg'}
    )
"""
fig_1_3.add_scattermapbox(
            lon=[100,48],
            lat=[80,50],
            mode = 'markers+text',
            below='',
            text = ['Russland:', 'USA'],  #a list of strings, one  for each geographical position  (lon, lat)
            marker_size=12, marker_color='rgb(235, 0, 100)')
            """

"\nfig_1_3.add_scattermapbox(\n            lon=[100,48],\n            lat=[80,50],\n            mode = 'markers+text',\n            below='',\n            text = ['Russland:', 'USA'],  #a list of strings, one  for each geographical position  (lon, lat)\n            marker_size=12, marker_color='rgb(235, 0, 100)')\n            "

In [19]:
fig_1_3.write_image("fig_1_3.png", width='1200', scale=1.5)

In [20]:
colors

['#1D7151',
 '#E39617',
 '#CC0000',
 '#E3DCCA',
 '#505C5D',
 '#8FBF99',
 '#F2C279',
 '#F2AA91',
 '#CCAD99',
 '#ACBFBF']

### Figure 1.4 Evolution of global GHG Emissions

In [21]:
#text
fig_1_4_title='Die gesamten Treibhausgasemissionen steigen.'
fig_1_4_description='''Bei prozentualer Ansicht scheint es, als würde der Anteil der <font color='#1D7151'>Lebensmittelindustrie</font> an den <font color='#CC0000'>globalen</font> Emissionen sinken. Die absolute Anzahl der Emissionen stagniert jedoch in den letzten 20 Jahren bei ungefähr 17 Milliarden Tonnen jährlich. <br>
                Dagegen steigt die Summe aller globalen Treibhausgasemissionen stetig an (2015 rund 50 Milliarden Tonnen). Der Anstieg kann u.a. auf ein hohes Bevölkerungswachstum und die andauernde Nutzung fossiler Brennstoffe zurückgeführt werden.
                '''

In [22]:
#figure
df_world_total_sub=df_world[df_world['Entity'].isin(['World'])]

#line-graph
fig_1_4 = go.Figure()
fig_1_4.add_trace(go.Scatter(x=df_world_total_sub['Year'], y=df_world_total_sub['total_ghg'],
                    mode='lines',
                    marker_color=colors[2],
                    name='Gesamt'))
fig_1_4.add_trace(go.Scatter(x=df_world_total_sub['Year'], y=df_world_total_sub['total_food_emissions'], marker_color=colors[0],
    mode='lines',
    name='Lebensmittelindustrie'))

fig_1_4.update_traces(mode="lines", hovertemplate='%{y:,r}')
fig_1_4.update_layout(hovermode="x unified",
    margin=dict(t=30, b=10, l=10, r=10),
    #legend={'orientation':'h', 'yanchor':"bottom",'y':1.01,'xanchor':"right",'x':1},
    font_family=font,
    title={'text':"Entwicklung der Treibhausgasemissionen der <b style='color:#1D7151'>Landwirtschaft</b> und <b style='color:#CC0000'>gesamt</b>",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1.0}
    )
fig_1_4.update_yaxes(rangemode="tozero")
fig_1_4.update_xaxes(tick0=1992, dtick=3)

In [23]:
fig_1_4.write_image("fig_1_4.png", width='800', scale=1.5)

## 2 Environmental Impact of single foods
### Figure 2.1 GHG-emissions, freshwater- and land-use of single foods

In [24]:
#text
fig_2_1_title='Welche Lebensmittel sind besonders umweltschädlich?'
fig_2_1_description='''Wichtige Faktoren um den negativen Einfluss von Lebensmitteln auf unsere Umwelt zu messen sind <b>Fläche, Süßwasserverbrauch</b> und <b>CO2-Emissionen</b>. Die Grafik zeigt diese Faktoren anhand ausgewählter Lebensmittel.
                    Zur besseren Vergleichbarkeit der einzelnen Lebensmittel beziehen sich alle Werte auf 1000 kcal des Produkts.<br>
                    Im Vergleich zu <font color='#1D7151'>pflanzlichen</font> Lebensmitteln, erzeugen <font color='#CC0000'>Fleisch und Fisch</font> deutlich mehr Emissionen. Zudem haben sie durch die Tierhaltung und Produktion von Futtermitteln einen höheren Flächen- und Süßwasserverbrauch.
                    '''

In [25]:
#preparation
df_impact_single = pd.read_csv('../Dashboard/fig4_effect_of_different_foods.csv')
df_impact_single['Entity']=df_impact_single['Entity'].astype('category')
df_impact_single['category']=df_impact_single['category'].astype('category')

#Drop rows with missing values
df_impact_single=df_impact_single.dropna()

#subset with selected foods
df_impact_single_sub1=df_impact_single[df_impact_single['Entity'].isin(['Apfel', 'Banane', 'Kartoffeln', 'Tofu', 'Rindfleisch (Milchvieh)', 'Milch', 'Rindfleisch (Fleischzucht)', 'Hühnerfleisch', 'Käse',  'Fisch', 
            'Gerste','Palmöl', 'Dunkle Schokolade', 'Kaffee'])]

In [26]:
#figure
# Create Scatterplot
fig_2_1=px.scatter(df_impact_single_sub1, x="ghg", y="land",
                    size="freshwater", color="category", 
                    #Define colors and order of legend
                    color_discrete_map={
                    "Pflanzliche Produkte": colors[0],
                    "Fleisch & Fisch": colors[2],
                    "Milchprodukte & Eier": colors[1],
                    "Süßes": colors[4],
                    "Getreide": colors[8],
                    "Getränke": colors[6],
                    "Öle & Fette": colors[5]},
                    category_orders={"category": ["Fleisch & Fisch", "Milchprodukte & Eier", "Pflanzliche Produkte", "Getreide", "Öle & Fette", "Süßes", 'Getränke']},
                    
                    text="Entity",
                    size_max=50, 
                    #In order to avoid jumping, the range is set to min/max of both axis
                    range_y =[df_impact_single['land'].min()-20, df_impact_single['land'].max()+10], range_x=[df_impact_single['ghg'].min()-5, df_impact_single['ghg'].max()+10]
                    )
#Update Styling Scatterplot

#Axis labels
fig_2_1.update_layout(
            xaxis = {"title": "Treibhausgasemissionen pro 1000kcal in KG"},
            yaxis={'title':'Benötigte Fläche pro 1000kcal in m<sup>2</sup>'},
            font_family=font,
            legend_title_text=None,
            title={'text':"Umweltauswirkungen ausgewählter Lebensmittel",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10)
            )

#Disable Hover labels
fig_2_1.update_traces(mode='markers', hovertemplate=None, hoverinfo='skip')


#Add annotations
fig_2_1.add_annotation(x=1, y=7,
            text="Apfel, Kartoffeln, <br>Banane, Tofu, <br>Gerste, Palmöl",
            showarrow=True,
            ax=0,ay=-40,
            yshift=5)
fig_2_1.add_annotation(x=6.2, y=30,
            text="Käse",
            showarrow=True,
            ax=0, ay=-35,
            xshift=0)
fig_2_1.add_annotation(x=5, y=14.5,
            text="Milch",
            showarrow=True,
            ax=-20, ay=-70,
            xshift=0)
fig_2_1.add_annotation(x=5, y=6,
            text="Hühnerfleisch",
            showarrow=True,
            ax=-10,ay=30,
            xshift=0, yshift=-10)
fig_2_1.add_annotation(x=8, y=5,
            text="Fisch",
            showarrow=True,
            ax=20,ay=50,
            xshift=0)
fig_2_1.add_annotation(x=12.5, y=14,
            text="Rindfleisch<br>(Milchvieh)",
            showarrow=True,
            ax=15,ay=40,
            xshift=0)
fig_2_1.add_annotation(x=9, y=13,
            text="Dunkle<br>Schokolade",
            showarrow=True,
            ax=27,ay=-50)
fig_2_1.add_annotation(x=36.5, y=118,
            text="Rindfleisch (Fleischzucht)",
            showarrow=True,
            ax=0,ay=35,
            xshift=0)
fig_2_1.add_annotation(x=51, y=38.5,
            text="Kaffee",
            showarrow=True,
            ax=0,ay=-25)


In [27]:
fig_2_1.write_image("fig_2_1.png", width='1200', scale=1.5)

### Fig 2.2: Only plant products

In [28]:
#text
fig_2_2_title='Welche pflanzlichen Lebensmittel sind besonders umweltschädlich?'
fig_2_2_description='''Wie die vorhergehende Abbildung zeigt, sind pflanzliche Produkte generell weniger klimaschädlich als tierische Produkte. Dies gilt auch für Öle und Getreide wie z.B. Gerste, Mais und Reis.<br>
                    Betrachten wir diese Produkte genauer, sind <font color='#1D7151'>Tomaten</font> aufgrund des hohen Wasserbedarf am umweltschädlichsten. 
                    Die Werte der anderen Produkte liegen dicht im unteren Wertebereich beieinander.
                    '''

In [29]:
#figure

#subset with selected foods
df_impact_single_sub2=df_impact_single[df_impact_single['Entity'].isin(['Mais', 'Gerste', 'Tomate', 'Apfel', 'Banane', 'Nüsse', 'Kohl', 'Beeren', 'Zitrone', 'Olivenöl', 'Tofu', 'Kartoffeln', 'Reis'])]

# Create Scatterplot
fig_2_2=px.scatter(df_impact_single_sub2, x="ghg", y="land",
                    size="freshwater", color="category", 
                    #Define colors and order of legend
                    color_discrete_map={
                    "Pflanzliche Produkte": colors[0],
                    "Fleisch & Fisch": colors[2],
                    "Milchprodukte & Eier": colors[1],
                    "Süßes": colors[4],
                    "Getreide": colors[8],
                    "Getränke": colors[6],
                    "Öle & Fette": colors[5]},
                    category_orders={"category": ["Fleisch & Fisch", "Milchprodukte & Eier", "Pflanzliche Produkte", "Getreide", "Öle & Fette", "Süßes", 'Getränke']},
                    
                    #text="Entity",
                    size_max=50, 
                    #In order to avoid jumping, the range is set to min/max of both axis
                    range_y =[-1, 10], range_x=[-0.5,12]
                    )
#Update Styling Scatterplot

#Axis labels
fig_2_2.update_layout(
            xaxis = {"title": "Treibhausgasemissionen pro 1000kcal in KG"},
            yaxis={'title':'Benötigte Fläche pro 1000kcal in m<sup>2</sup>'},
            font_family=font,
            legend_title_text=None,
            title={'text':"Umweltauswirkungen <b style='color:#1D7151'>pflanzlicher</b> Lebensmittel",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10)
            )

#Disable Hover labels
fig_2_2.update_traces(mode='markers', hovertemplate=None, hoverinfo='skip')


#Add Annotations
fig_2_2.add_annotation(x=11, y=5.5, text='Tomate', showarrow=False, ax=0,ay=0)
fig_2_2.add_annotation(x=2.8, y=5.1, text='Beeren', showarrow=False, ax=0,ay=0)
fig_2_2.add_annotation(x=3.2, y=2.5, text='Kohl', showarrow=False, ax=0,ay=0)
fig_2_2.add_annotation(x=1.2, y=0.7, text='Reis', showarrow=True, ax=50,ay=10)
fig_2_2.add_annotation(x=1.16, y=1.3, text='Tofu', showarrow=True, ax=20,ay=-20)
fig_2_2.add_annotation(x=0.22, y=0.2, text='Gerste', showarrow=True, ax=0,ay=30)
fig_2_2.add_annotation(x=0.37, y=0.65, text='Mais', showarrow=True, ax=-25,ay=0)
fig_2_2.add_annotation(x=0.89, y=1.3, text='Apfel', showarrow=True, ax=0,ay=-35)
fig_2_2.add_annotation(x=0.62, y=1.2, text='Kartoffeln', showarrow=True, ax=15,ay=50)
fig_2_2.add_annotation(x=0.06, y=2.1, text='Nüsse', showarrow=False, ax=0,ay=0)
fig_2_2.add_annotation(x=0.6, y=3.0, text='Olivenöl', showarrow=True, ax=0,ay=-60)
fig_2_2.add_annotation(x=1.45, y=3.2, text='Banane', showarrow=True, ax=0,ay=-30)
fig_2_2.add_annotation(x=1.3, y=2.6, text='Zitrone', showarrow=True, ax=35,ay=0)

In [30]:
fig_2_2.write_image("fig_2_2.png", width='800', scale=1.5)

### Fig 2.3: Only animal products

In [31]:
#text
fig_2_3_title='Welche tierischen Lebensmittel sind besonders umweltschädlich?'
fig_2_3_description='''Betrachten wir nun ausschließlich tierische Produkte, fällt auf, dass sich die einzelnen Lebensmittel hier stärker unterscheiden. Zudem ist der Wertebereich wieder größer. <br>
                <font color='#CC0000'>Garnelen</font> zeichnen sich durch vergleichbar hohe Treibhausgasemissionen, einen sehr hohen Süßwasser- und geringeren Flächenverbrauch aus. <br>
                <font color='#CC0000'>Rindfleisch aus Fleischzucht</font> ist aufgrund der Viehhaltung und der benötigten Anbaufläche besonders flächenintensiv und erzeugt die meisten Treibhausgase. <br>
                <font color='#E39617'>Eier und Milchprodukte</font> sind zwar verglichen mit Fleisch & Fisch weniger klimaschädlich, liegen jedoch immer noch deutlich über den Werten von pflanzlichen Produkten.
                '''

In [32]:
#figure

#subset with selected foods
df_impact_single_sub3=df_impact_single[df_impact_single['category'].isin(['Fleisch & Fisch', 'Milchprodukte & Eier'])]

# Create Scatterplot
fig_2_3=px.scatter(df_impact_single_sub3, x="ghg", y="land",
                    size="freshwater", color="category", 
                    #Define colors and order of legend
                    color_discrete_map={
                    "Pflanzliche Produkte": colors[0],
                    "Fleisch & Fisch": colors[2],
                    "Milchprodukte & Eier": colors[1],
                    "Süßes": colors[4],
                    "Getreide": colors[8],
                    "Getränke": colors[6],
                    "Öle & Fette": colors[5]},
                    category_orders={"category": ["Fleisch & Fisch", "Milchprodukte & Eier", "Pflanzliche Produkte", "Getreide", "Öle & Fette", "Süßes", 'Getränke']},
                    
                    text="Entity",
                    size_max=50, 
                    #In order to avoid jumping, the range is set to min/max of both axis
                    range_y =[df_impact_single['land'].min()-20, df_impact_single['land'].max()+10], range_x=[df_impact_single['ghg'].min()-5, df_impact_single['ghg'].max()+10]
                    )
#Update Styling Scatterplot

#Axis labels
fig_2_3.update_layout(
            xaxis = {"title": "Treibhausgasemissionen pro 1000kcal in KG"},
            yaxis={'title':'Benötigte Fläche pro 1000kcal in m<sup>2</sup>'},
            font_family=font,
            legend_title_text=None,
            title={'text':"Umweltauswirkungen <b style='color:#CC0000'>tierischer</b> Lebensmittel",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10)
            )

#Disable Hover labels
fig_2_3.update_traces(mode='markers', hovertemplate=None, hoverinfo='skip')

#Add annotations
fig_2_3.add_annotation(x=6.2, y=30, text="Käse", showarrow=True, ax=0, ay=-35, xshift=0)
fig_2_3.add_annotation(x=5, y=14.5, text="Milch", showarrow=True,ax=-20, ay=-70, xshift=0)
fig_2_3.add_annotation(x=5, y=6,text="Hühner-<br>fleisch",showarrow=True,ax=-10,ay=30,xshift=0, yshift=-10)
fig_2_3.add_annotation(x=5, y=7,text="Schweine-<br>fleisch", showarrow=True, ax=-50,ay=-30, xshift=0)
fig_2_3.add_annotation(x=8, y=5, text="Fisch", showarrow=True, ax=20,ay=50, xshift=0)
fig_2_3.add_annotation(x=12.5, y=14, text="Rindfleisch<br>(Milchvieh)", showarrow=True, ax=25,ay=30, xshift=0)
fig_2_3.add_annotation(x=36.5, y=110, text="Rindfleisch (Fleischzucht)", showarrow=False, ax=0,ay=35, xshift=0)
fig_2_3.add_annotation(x=12.5, y=108, text="Lammfleisch", showarrow=False, ax=0,ay=35, xshift=0)
fig_2_3.add_annotation(x=26, y=22, text="Garnele", showarrow=False, ax=0,ay=-35, xshift=0)
fig_2_3.add_annotation(x=3, y=4, text="Eier", showarrow=True, ax=-30,ay=0, xshift=0)

In [33]:
fig_2_3.write_image("fig_2_3.png", width='800', scale=1.5)

## 3 Comparing different diets
### Figure 3.1 Comparing diets of different countries to the sustainable diet recommended by EAT-Lancet

In [34]:
#text
fig_3_1_title='Wie sieht eine gesunde und nachhaltige Ernährung aus?'
fig_3_1_description='''Die EAT-Lancet-Kommission ist ein Zusammenschluss aus 37 internationalen und weltweit renommierten Forscher*Innen. Sie beschäftigen sich mit der Frage, 
                    inwieweit trotz rasant wachsenden Bevölkerungszahlen allen Menschen eine <b>gesunde und nachhaltige Ernährung</b> ermöglicht werden kann. 
                    Gemeinsam haben sie eine Ernährungsweise entwickelt, die empfohlen wird, um diese Ziele zu erreichen. Das bedeutet, wenn sich alle Menschen an dieser Ernährungsweise orientieren würden, könnten wir der gesamten Weltbevölkerung eine gesunde Ernährung mit ökologischer Nachhaltigkeit ermöglichen. <br>
                    Vergleichen wir diese Ernährungsweise mit den Durchschnittswerten ausgewählter Länder, zeigt sich zunächst, dass v.a. die Industriestaaten insgesamt zu viel Lebensmittel konsumieren.<br>
                    Ein weiterer wichtiger Unterschied ist die Zusammensetzung der Ernährung. Aktuell konsumieren die Länder deutlich mehr <b>tierische Produkte</b> als empfohlen. Durchschnittlich isst jeder Deutsche knapp vier mal so viel <font color='#CC0000'>Fleisch und Fisch</font> als empfohlen (269g vs. 71g), in den USA ist es sogar über fünf-mal so viel. 
                    Ähnlich verhält es sich bei <font color='#E39617'>Milchprodukten und Eiern</font>. Hier ist der Anteil in Deutschland knapp drei mal größer als empfohlen. <br>
                    Das zeigt, dass wir, um uns der empfohlenen Ernährungsweise anzunähern, insbesondere den Konsum von Fleisch und tierischen Produkten reduzieren müssen.
                    '''

In [35]:
#preparation df
df_diet = pd.read_csv('../Dashboard/fig5_diets.csv')
df_diet['country']=df_diet['country'].astype('category')
df_diet['food']=df_diet['food'].astype('category')
df_diet['group']=df_diet['group'].astype('category')

In [36]:
#figure
df_diet_sub=df_diet.groupby(['country', 'group']).agg({
                        'gramm':'sum'
                        }).reset_index(level=[0,1])
df_diet_sub=df_diet_sub[df_diet_sub['country'].isin(['EAT-Lancet','Deutschland', 'Vereinigte Staaten von Amerika', 'Frankreich', 'Spanien', 'Italien'])]
fig_3_1=px.bar(data_frame=df_diet_sub, x="gramm",y="country",
        color="group", text='gramm', orientation='h', 
        color_discrete_map={
                    "Pflanzliche Produkte": colors[0],
                    "Fleisch & Fisch": colors[2],
                    "Milchprodukte & Eier": colors[1],
                    "Zucker": colors[4],
                    "Getreide": colors[8],
                    "Öl": colors[5]},
        category_orders={"group": ["Fleisch & Fisch","Milchprodukte & Eier",  "Pflanzliche Produkte", "Getreide", "Öl", "Zucker"], 'country':['Vereinigte Staaten von Amerika', 'Italien', 'Frankreich', 'Spanien', 'Deutschland', 'EAT-Lancet']},
        )

#Axis labels
fig_3_1.update_layout(
            xaxis = {"title": "Lebensmittelkategorie in Gramm"},
            yaxis={'title':None},
            font_family=font,
            legend_title_text=None,
            #Minimum text size for inside text
            uniformtext=dict(minsize=13,
            mode='hide'),
            yaxis_tickformat = '',
            title={'text':"Durchschnittliche Ernährungszusammensetzung ausgewählter Länder und die Empfehlung der EAT-Lancet Kommission",'font_family':'Montserrat-Regular, Raleway, sans-serif','x':0, 'y':1},
            margin=dict(t=30, b=10, l=10, r=10)
            
            )

fig_3_1.update_traces(hovertemplate=None, hoverinfo='skip',
        #Text formatting
        texttemplate="%{x:.0f} g",
        textposition='inside',
        insidetextanchor='middle',
        textangle=0)

In [37]:
fig_3_1.write_image("fig_3_1.png", width='1200', scale=1.5)

## 4 Conclusion

In [38]:
conclusion='''Die Lebensmittelindustrie ist mitverantwortlich für verschiedene Umweltschäden z.B. Treibhausgasemissionen, Ressourcenverbrauch und Eutrophierung.<br>
            Die Emissionen stammen hauptsächlich aus der Landwirtschaftlichen Produktion und der Bewirtung der Flächen. Nur ein geringer Anteil entsteht bei der Verpackung und dem Transport.<br>
            Abgesehen von einigen, wenigen Ländern, in denen durch einen großen Agrarsektor und eine hohe Bevölkerungszahl überdurchschnittlich viele Emissionen entstehen, verteilen sich die globalen Emissionen der Lebensmittelindustrieweltweit ähnlich.<br>
            Einzelne Lebensmittel tragen unterschiedlich stark zum Umwelteinfluss der Industrie bei. <b><font color='#1D7151'>Pflanzliche Produkte</font></b> produzieren deutlich weniger Treibhausgase und haben einen geringeren Ressourcenverbrauch als <b><font color='#cc0000'>tierische Produkte</font></b> wie Fleisch und Milchprodukte.
            Verglichen mit der empfohlenen Ernährungsweise konsumieren die Industriestaaten deutlich <b>zu viel Fleisch, Fisch und tierische Produkte</b>. Um global eine gesunde und nachhaltige Ernährung zu ermöglichen, sollten diese Kategorien reduziert werden.<br>
            Was wir essen hat folglich einen großen Einfluss auf unsere Umwelt. Ein Verzicht auf Fleisch und Fisch ist ein guter und effektiver Weg, um mehr Nachhaltigkeit in den eigenen Alltag zu integrieren.
            Gleichzeitig unterstützt dies eine gesunde Ernährung und reduziert das Risiko von Krankheiten wie Diabetes oder Schlaganfällen. Nicht zuletzt kommt eine pflanzenbasierte Ernährung vor allem aber auch den Tieren zu Gute.
            '''

## Render text and figures in template

In [39]:
output = template.render(introduction=introduction, 
                        fig_1_1_title=fig_1_1_title, fig_1_1_description=fig_1_1_description,
                        fig_1_2_title=fig_1_2_title, fig_1_2_description=fig_1_2_description, 
                        fig_1_3_title=fig_1_3_title, fig_1_3_description=fig_1_3_description, 
                        fig_1_4_title=fig_1_4_title, fig_1_4_description=fig_1_4_description, 
                        fig_2_1_title=fig_2_1_title, fig_2_1_description=fig_2_1_description,
                        fig_2_2_title=fig_2_2_title, fig_2_2_description=fig_2_2_description, 
                        fig_2_3_title=fig_2_3_title, fig_2_3_description=fig_2_3_description,  
                        fig_3_1_title=fig_3_1_title, fig_3_1_description=fig_3_1_description, 
                        conclusion=conclusion
                        )

In [40]:
# Write the HTML file
with open('report.html', 'w') as f:
    f.write(output)

In [43]:
#convert html to pdf
with open('report.pdf', "w+b") as result_file:
    pisa.CreatePDF(
        src=output,  # HTML to convert
        dest=result_file)  # File handle to receive result

Selector name or qualifier expected:: ('', '"@font-face {\\n  fon')
